# Load Data

In [1]:
# %load_ext autoreload
# %autoreload 2

In [1]:
# !pip install polars

In [4]:
!pip install implicit >> _

In [13]:
# !rm -rf /kaggle/working/*

In [2]:
import gdown

url = 'https://drive.google.com/uc?id=1GcfpQXP9t0sfwNTPaNYDi94VSBnjxiOe'
output = 'cat_features.pq'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1HD7cuZJVeo3YcoZ2yH_lX-oeR0hsX6dx'
output = 'clickstream.pq'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1rvtZHHd-EEssfEbWrIVWS4r2w7Ykmwmd'
output = 'events.pq'
gdown.download(url, output, quiet=False)

url = 'https://drive.google.com/uc?id=1CtZNq0fKzIp4y3qpvSmklkS5pAbJvCoI'
output = 'text_features.pq'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1GcfpQXP9t0sfwNTPaNYDi94VSBnjxiOe
From (redirected): https://drive.google.com/uc?id=1GcfpQXP9t0sfwNTPaNYDi94VSBnjxiOe&confirm=t&uuid=5f6be30a-9e6b-4887-8b5d-058f4ee3652d
To: /kaggle/working/cat_features.pq
100%|██████████| 633M/633M [00:06<00:00, 99.7MB/s] 
Downloading...
From (original): https://drive.google.com/uc?id=1HD7cuZJVeo3YcoZ2yH_lX-oeR0hsX6dx
From (redirected): https://drive.google.com/uc?id=1HD7cuZJVeo3YcoZ2yH_lX-oeR0hsX6dx&confirm=t&uuid=e0a34ee3-a30f-435f-ac2d-717f32a7d1ad
To: /kaggle/working/clickstream.pq
100%|██████████| 968M/968M [00:08<00:00, 115MB/s]  
Downloading...
From: https://drive.google.com/uc?id=1rvtZHHd-EEssfEbWrIVWS4r2w7Ykmwmd
To: /kaggle/working/events.pq
100%|██████████| 972/972 [00:00<00:00, 3.17MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1CtZNq0fKzIp4y3qpvSmklkS5pAbJvCoI
From (redirected): https://drive.google.com/uc?id=1CtZNq0fKzIp4y3qpvSmklkS5pAbJvCoI&confirm=t&uui

'text_features.pq'

In [5]:
from datetime import timedelta
import polars as pl
import implicit

In [6]:
DATA_DIR = '/kaggle/working'

# df_test_users = pl.read_parquet(f'{DATA_DIR}/test_users.pq')
df_clickstream = pl.read_parquet(f'{DATA_DIR}/clickstream.pq')

df_cat_features = pl.read_parquet(f'{DATA_DIR}/cat_features.pq')
# df_text_features = pl.read_parquet(f'{DATA_DIR}/text_features.pq')
df_event = pl.read_parquet(f'{DATA_DIR}/events.pq')

In [7]:
df_clickstream.head()

cookie,item,event,event_date,platform,surface,node
i64,i64,i64,datetime[ns],i64,i64,u32
0,19915558,17,2025-02-05 02:30:59,3,2,115659
0,2680232,17,2025-01-24 21:16:57,3,2,115829
1,4247649,17,2025-01-29 23:00:58,2,2,7
1,4247649,17,2025-02-17 14:55:17,2,2,7
1,2171135,17,2025-01-17 19:23:29,2,2,214458


In [8]:
df_event.head()

event,is_contact
i64,i64
0,1
11,0
4,1
16,0
15,1


In [9]:
df_cat_features.head()

item,location,category,clean_params,node
i64,i64,i64,str,u32
9,8385,57,"""[{""attr"":1157,""value"":664427},…",194747
17,2707,35,"""[{""attr"":2140,""value"":501466},…",352905
144,8383,8,"""[{""attr"":802,""value"":35791},{""…",17188
202,5397,57,"""[{""attr"":1157,""value"":490527},…",194766
236,2105,64,"""[{""attr"":112,""value"":420797},{…",153951


In [10]:
df_cat_features = df_cat_features[['item', 'location', 'category' , 'node']]

# PREPARE TRAIN EVAL

In [11]:
EVAL_DAYS_TRESHOLD = 14
CAND_DAYS_TRESHOLD = 21

In [12]:
treshhold = df_clickstream['event_date'].max() - timedelta(days=EVAL_DAYS_TRESHOLD)
cand_treshhold = df_clickstream['event_date'].max() - timedelta(days=CAND_DAYS_TRESHOLD)

In [13]:
cand_treshhold

datetime.datetime(2025, 2, 2, 0, 0)

In [14]:
treshhold

datetime.datetime(2025, 2, 9, 0, 0)

In [15]:
df_clickstream = df_clickstream.join(df_cat_features['item', 'location', 'category'], on = ['item'])

In [16]:
df_clickstream = df_clickstream.with_columns(
    pl.col('event').is_in(
        df_event.filter(
            pl.col('is_contact')==1
        )['event'].unique()
    ).alias('is_target')
)

In [17]:
df_train = df_clickstream.filter(df_clickstream['event_date'] <= treshhold)
df_eval = df_clickstream.filter(df_clickstream['event_date'] > treshhold)[['cookie', 'node', 'event', 'is_target']]

In [18]:
df_ranker = df_train.filter(df_train['event_date'] > cand_treshhold)
df_cand = df_train.filter(df_train['event_date'] <= cand_treshhold)

df_ranker = df_ranker.filter(pl.col('cookie').is_in(df_cand['cookie'].unique()))

In [19]:
df_eval = df_eval.join(df_train, on=['cookie', 'node'], how='anti')

In [20]:
df_eval = df_eval.filter(
    pl.col('is_target')
)

In [21]:
df_eval = df_eval.filter(
    pl.col('cookie').is_in(df_train['cookie'].unique())
).filter(
    pl.col('node').is_in(df_train['node'].unique())
)

In [22]:
df_eval = df_eval.unique(['cookie', 'node'])

# TRAIN MODEL

## ALS

In [23]:
def get_als_pred(users, nodes, user_to_pred, N):
    user_ids = users.unique().to_list()
    item_ids = nodes.unique().to_list()
        
    user_id_to_index = {user_id: idx for idx, user_id in enumerate(user_ids)}
    item_id_to_index = {item_id: idx for idx, item_id in enumerate(item_ids)}
    index_to_item_id = {v:k for k,v in item_id_to_index.items()}
    
    rows = users.replace_strict(user_id_to_index).to_list()
    cols = nodes.replace_strict(item_id_to_index).to_list()
    
    values = [1] * len(users)
    
    sparse_matrix = csr_matrix((values, (rows, cols)), shape=(len(user_ids), len(item_ids)))
    
    model = implicit.als.AlternatingLeastSquares(iterations=10, factors=60)
    model.fit(sparse_matrix, )
    
    
    user4pred = np.array([user_id_to_index[i] for i in user_to_pred])
    
    recommendations, scores = model.recommend(user4pred, sparse_matrix[user4pred], N=N, filter_already_liked_items=True)
    
    df_pred = pl.DataFrame(
        {
            'node': [
                [index_to_item_id[i] for i in i] for i in recommendations.tolist()
            ], 
             'cookie': list(user_to_pred),
            'scores': scores.tolist()
            
        }
    )
    df_pred = df_pred.explode(['node', 'scores'])
    return df_pred

In [24]:
N = 300

In [25]:
from scipy.sparse import csr_matrix
import numpy as np
import implicit


users = df_cand["cookie"]
nodes = df_cand["node"]
pred_users = df_ranker['cookie'].unique().to_list()


df_ranker_pred = get_als_pred(users, nodes, pred_users, N=N)

/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: Intel MKL BLAS is configured to use 2 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'MKL_NUM_THREADS=1' or by callng 'threadpoolctl.threadpool_limits(1, "blas")'. Having MKL use a threadpool can lead to severe performance issues
  check_blas_config()
/usr/local/lib/python3.11/dist-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
df_ranker_pred.shape

(31526700, 3)

In [29]:
df_ranker_pred.head()

node,cookie,scores
i64,i64,f64
115834,0,0.807968
116118,0,0.743365
115704,0,0.719981
116123,0,0.71014
116119,0,0.632979


In [30]:
users = df_train["cookie"]
nodes = df_train["node"]

pred_users = df_eval['cookie'].unique().to_list()

df_pred = get_als_pred(users, nodes, pred_users, N=N)

  0%|          | 0/10 [00:00<?, ?it/s]

# Second stage

In [31]:
import catboost as cb
from catboost import Pool

In [32]:
df_targets = df_ranker[['cookie', 'node', 'is_target']].sort(by = ['is_target'], descending=True).unique(['cookie', 'node' ])
df_ranker_pred = df_ranker_pred.join(df_targets, on = ['cookie', 'node'], how = 'left').with_columns(pl.col('is_target').cast(int))

### feature engineering

In [33]:
df_cookie_category_features = df_cand.group_by(['cookie', 'category']).agg(
    [
        pl.col('is_target').sum().alias('num_contacts'),
        pl.col('is_target').count().alias('num_events'),
        pl.col('surface').unique().alias('surface_unique_counts'),
        pl.col('location').unique().alias('location_unique_counts')
    ]
).with_columns(
    [
        pl.col('surface_unique_counts').list.len(),
        pl.col('location_unique_counts').list.len()
    ]
)

In [34]:
def get_cat_features(df):
    return df.join(
        df_cat_features['node', 'category'].unique(), 
        on = ['node']
    ).join(
        df_cookie_category_features, on =['category', 'cookie'], how='left'
    )

In [35]:
df_ranker_pred = get_cat_features(df_ranker_pred)
df_pred = get_cat_features(df_pred)

In [36]:
df_ranker_pred = df_ranker_pred.with_columns(
    pl.col('is_target').fill_null(0)
)

In [37]:
df_ranker_pred

node,cookie,scores,is_target,category,num_contacts,num_events,surface_unique_counts,location_unique_counts
i64,i64,f64,i64,i64,u32,u32,u32,u32
115834,0,0.807968,0,19,4,81,6,3
116118,0,0.743365,0,19,4,81,6,3
115704,0,0.719981,0,19,4,81,6,3
116123,0,0.71014,0,19,4,81,6,3
116119,0,0.632979,0,19,4,81,6,3
…,…,…,…,…,…,…,…,…
1911,149999,0.118499,0,25,0,1,1,1
170546,149999,0.117799,0,52,null,null,null,null
115859,149999,0.117552,0,19,0,7,1,2


In [38]:
cat_features = None
features = ['scores', 'num_contacts', 'category', 'num_events', 'surface_unique_counts', 'location_unique_counts']

In [39]:
def get_pool(df):
    df = df.sort(by = ['cookie', 'node'])
    
    group_id = df['cookie'].to_list()
    y = df['is_target'].to_list()
    
    pool = Pool(
        data=df[features].to_pandas(),
        group_id=group_id, 
        label = y, 
        cat_features=cat_features,
    )
    return pool

In [40]:
all_users = df_ranker_pred['cookie'].unique().to_list()

In [41]:
import random
random.shuffle(all_users)

In [42]:
num_eval = int(len(all_users)*0.1)

In [43]:
eval_users, test_users, train_users = all_users[:num_eval], all_users[num_eval: 2*num_eval], all_users[2*num_eval:]

In [44]:
eval_pool = get_pool(df_ranker_pred.filter(pl.col('cookie').is_in(eval_users)))
test_pool = get_pool(df_ranker_pred.filter(pl.col('cookie').is_in(test_users)))
train_pool = get_pool(df_ranker_pred.filter(pl.col('cookie').is_in(train_users)))

In [45]:
# df_ranker_pred['is_target'].sum()

In [46]:
train_pool.shape, test_pool.shape, eval_pool.shape

((25221900, 6), (3152400, 6), (3152400, 6))

In [47]:
params = {
  "boosting_type": "Plain",
  "devices": "0",
  "early_stopping_rounds": 100,
  "eval_metric": "RecallAt:top=40",
  "learning_rate": 0.1,
  "max_ctr_complexity": 1,
  "nan_mode": "Min",
  "num_trees": 2000,
  "objective": "PairLogitPairwise",
  "random_state": 42,
  "task_type": "GPU"
}

ranker = cb.CatBoost(params = params)

In [48]:
ranker.fit(X=train_pool, metric_period=100, eval_set=eval_pool)

Metric RecallAt:top=40 is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.8774049	test: 0.8741262	best: 0.8741262 (0)	total: 2.36s	remaining: 1h 18m 43s
100:	learn: 0.9020588	test: 0.9005624	best: 0.9005942 (97)	total: 20.3s	remaining: 6m 22s
200:	learn: 0.9029017	test: 0.9005593	best: 0.9006100 (107)	total: 38.3s	remaining: 5m 42s
300:	learn: 0.9034517	test: 0.8999764	best: 0.9010747 (208)	total: 56.3s	remaining: 5m 17s
bestTest = 0.9010747395
bestIteration = 208
Shrink model to first 209 iterations.


In [49]:
ranker.eval_metrics(test_pool, metrics=["RecallAt:top=40"],ntree_start=ranker.tree_count_-1 )

{'RecallAt:top=40': [0.9012165009524165]}

Этот Recall@40 не показывает настоящее качество, так как считается на отобранных кандидатах, а не на всем датасете

In [50]:
df_pred = df_pred.with_columns(
    catboost_score = ranker.predict(df_pred.to_pandas())
)

# CALC EVAL METRICS

In [54]:
def recall_at(df_true, df_pred, k=40):
    return  df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']], 
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()


def calc_metrics(df_true, df_pred, k=40):
    recall_at_k = df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']],
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/pl.col(
                'value'
            ).count()
        ]
    )['value'].mean()

    precision_at_k = df_true[['node', 'cookie']].join(
        df_pred.group_by('cookie').head(k).with_columns(value=1)[['node', 'cookie', 'value']],
        how='left',
        on = ['cookie', 'node']
    ).select(
        [pl.col('value').fill_null(0), 'cookie']
    ).group_by(
        'cookie'
    ).agg(
        [
            pl.col('value').sum()/k
        ]
    )['value'].mean()

    f1_at_k = 2 * (precision_at_k * recall_at_k) / (precision_at_k + recall_at_k)

    return recall_at_k, precision_at_k, f1_at_k

In [52]:
recall_at(df_eval, df_pred.sort(by=['cookie', 'scores'], descending=True), k=40)

0.15280514385876612

In [53]:
recall_at(df_eval, df_pred.sort(by=['cookie', 'catboost_score'], descending=True), k=40)

0.16215512535899404

In [57]:
import pandas as pd


pd.DataFrame(
    [
        [0.058067, 0.002983, 0.005675],
        calc_metrics(df_eval, df_pred.sort(by=['cookie', 'scores'], descending=True), k=40),
        calc_metrics(df_eval, df_pred.sort(by=['cookie', 'catboost_score'], descending=True), k=40),
    ],
    columns=['recall_at_40', 'precision_at_40', 'f1_at_40'],
    index=['most_popular', 'ASL', 'two-level_model']
)

,recall_at_40,precision_at_40,f1_at_40
most_popular,0.058067,0.002983,0.005675
ASL,0.152805,0.009290,0.017516
two-level_model,0.162155,0.009994,0.018828


Двухэтапная модель (отбор кандидатов, затем ранжирование кандидатов) увеличило метрики рекомендаций по сравнению с бейзлайном и колаборативной фильтрацией